# Political success, May 2021

In [1]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
from linearmodels import IV2SLS
%run Table_Summary.ipynb

In [4]:
data = pandas.read_csv(r'D:/BP/Data/APA_ASA.txt',sep='\t')
data['MAR'] = pandas.to_numeric(data.MAR, errors='coerce')
data['MRR'] = 100 - 100*data['MAR']
data['Subscriptions'] = data['Printed_Subsc_Institutional']
data['Journal'] = data.groupby(['ID'])[['Journal']].fillna(method='ffill')
data['ASA'] = data.groupby(['ID'])[['ASA']].fillna(method='ffill')
data = data[~((data.Journal=='Contexts') | (data.Journal=='Rose'))]
data = data[['Journal','Year','MRR','JIF','Subscriptions','Citable_Items','JIF_Percentile','ASA']]
data = data.set_index(['Journal','Year']).sort_index()
data

MRR    JIF  Subscriptions  Citable_Items  \
Journal               Year                                              
American Psychologist 2004  85.2  5.494         1938.0           36.0   
                      2005  76.4  6.460         1599.0           46.0   
                      2006  74.8  7.829         1532.0           46.0   
                      2007  84.5  6.967         1330.0           48.0   
                      2008  77.8  7.106         1219.0           34.0   
...                          ...    ...            ...            ...   
Teaching Sociology    2012  80.9    NaN         2176.0            NaN   
                      2013  84.0  0.980         2892.0           33.0   
                      2014  85.3  0.464         2995.0           26.0   
                      2015   NaN  0.559         3049.0           24.0   
                      2016   NaN  0.480            NaN           19.0   

                            JIF_Percentile  ASA  
Journal               Year                       
American Psychologist 2004          95.500  0.0  
                      2005          96.535  0.0  
                      2006          95.455  0.0  
                      2007          96.569  0.0  
                      2008          96.535  0.0  
...                                    ...  ...  
Teaching Sociology    2012             NaN  1.0  
                      2013          57.609  1.0  
                      2014          26.408  1.0  
                      2015          29.930  1.0  
                      2016          21.329  1.0  

[516 rows x 6 columns]

## Variables

In [15]:
mydf = data.copy()
mydf = mydf.reset_index()
mydf['SA']=mydf.groupby(['Year']).JIF.transform('mean')
mydf['SA']=mydf['SA'].rolling(window=5,min_periods=1).mean()
mydf['JIF']=mydf.groupby(['Journal'])['JIF'].rolling(window=5,min_periods=1).mean().reset_index(0,drop=True)
mydf['distance']=mydf['JIF']-mydf['SA']
mydf.loc[mydf.distance<0,'Political Success']=mydf['SA']
mydf.loc[mydf.distance>0,'Political Success']=mydf['JIF']
mydf['Subscriptions']=numpy.log(mydf['Subscriptions'])
mydf['JIF Percentile'] = mydf.groupby('Year')['JIF_Percentile'].transform(pandas.qcut,2,labels=False,duplicates='drop')
mydf['Citable Items'] = mydf.groupby('Year')['Citable_Items'].transform(pandas.qcut,2,labels=False,duplicates='drop')
mydf

,Journal,Year,MRR,JIF,Subscriptions,Citable_Items,JIF_Percentile,ASA,SA,distance,Political Success,JIF Percentile,Citable Items
0,American Psychologist,2004,85.2,5.494000,7.569412,36.0,95.500,0.0,2.907630,2.586370,5.494000,1.0,0.0
1,American Psychologist,2005,76.4,5.977000,7.377134,46.0,96.535,0.0,2.829190,3.147810,5.977000,1.0,0.0
2,American Psychologist,2006,74.8,6.594333,7.334329,46.0,95.455,0.0,2.890892,3.703441,6.594333,1.0,0.0
3,American Psychologist,2007,84.5,6.687500,7.192934,48.0,96.569,0.0,2.899298,3.788202,6.687500,1.0,0.0
4,American Psychologist,2008,77.8,6.771200,7.105786,34.0,96.535,0.0,2.984611,3.786589,6.771200,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,Teaching Sociology,2012,80.9,0.663500,7.685244,NaN,NaN,1.0,3.300109,-2.636609,3.300109,NaN,NaN
512,Teaching Sociology,2013,84.0,0.781000,7.969704,33.0,57.609,1.0,3.319557,-2.538557,3.319557,0.0,0.0
513,Teaching Sociology,2014,85.3,0.722000,8.004700,26.0,26.408,1.0,3.383563,-2.661563,3.383563,0.0,0.0
514,Teaching Sociology,2015,NaN,0.667667,8.022569,24.0,29.930,1.0,3.398727,-2.731061,3.398727,0.0,0.0


## Preparing data for regression

In [16]:
XY = mydf.copy()
XY = XY[['MRR','Political Success','Subscriptions','JIF Percentile','Citable Items','Journal','Year']]
XY = XY.dropna()
X1 = XY[['Political Success','Subscriptions','JIF Percentile','Citable Items','Journal','Year']]
X1 = pandas.get_dummies(X1,columns=['Journal','Year'])
X1 = X1.drop(['Journal_American Psychologist','Year_2004'],axis=1)
X1['Political Success'] = X1['Political Success']-X1['Political Success'].mean()
X1['Subscriptions'] = X1['Subscriptions']-X1['Subscriptions'].mean()
X1['Political Success*Subscriptions'] = X1['Political Success'] * X1['Subscriptions']
X1['Intercept'] = 1
X0 = X1.drop(['Political Success*Subscriptions'],axis=1)

## Regression

In [17]:
r0 = []
r1 = []
window = 5
reg0 = sm.OLS(XY['MRR'],X0).fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':window**2})
r0.append((reg0.params,reg0.bse,reg0.pvalues,reg0.rsquared_adj))
print(reg0.summary())
reg1 = sm.OLS(XY['MRR'],X1).fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':window**2})
r1.append((reg1.params,reg1.bse,reg1.pvalues,reg1.rsquared_adj))
reg1.summary()

                            OLS Regression Results                            
Dep. Variable:                    MRR   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                -2.232e+14
Date:                Wed, 28 Apr 2021   Prob (F-statistic):               1.00
Time:                        06:19:06   Log-Likelihood:                -1431.1
No. Observations:                 434   AIC:                             2970.
Df Residuals:                     380   BIC:                             3190.
Df Model:                          53                                         
Covariance Type:         hac-groupsum                                         
                                                             coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------

c:\users\ainhoa.urtasun\.myvirtualenv\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 53, but rank is 13
  warnings.warn('covariance of constraints does not have full '
c:\users\ainhoa.urtasun\.myvirtualenv\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 54, but rank is 14
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    MRR   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                -1.861e+14
Date:                Wed, 28 Apr 2021   Prob (F-statistic):               1.00
Time:                        06:19:06   Log-Likelihood:                -1431.0
No. Observations:                 434   AIC:                             2972.
Df Residuals:                     379   BIC:                             3196.
Df Model:                          54                                         
Covariance Type:         hac-groupsum                                         
==========================================================================================================================
                                                             coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Political Success                                          0.8835      0.148      5.968      0.000       0.593       1.174
Subscriptions                                             -0.7007      0.276     -2.540      0.011      -1.241      -0.160
JIF Percentile                                             0.6577      0.516      1.275      0.202      -0.353       1.668
Citable Items                                             -1.8575      0.551     -3.369      0.001      -2.938      -0.777
Journal_American Sociological Review                      17.7525      0.898     19.759      0.000      15.992      19.513
Journal_Behavioral Neuroscience                          -17.3176      3.100     -5.587      0.000     -23.393     -11.242
Journal_Contemporary Sociology                            -8.4756      2.008     -4.220      0.000     -12.412      -4.539
Journal_Developmental Psychology                           1.0653      0.893      1.193      0.233      -0.685       2.815
Journal_Emotion                                           -1.5373      3.411     -0.451      0.652      -8.223       5.148
Journal_Experimental and Clinical Psychopharmacology     -32.3552      2.763    -11.711      0.000     -37.770     -26.940
Journal_Health Psychology                                  7.1586      1.063      6.733      0.000       5.075       9.243
Journal_History of Psychology                            -14.0454      1.450     -9.686      0.000     -16.888     -11.203
Journal_JEP: Animal Behavior Processes                   -19.6511      1.669    -11.777      0.000     -22.921     -16.381
Journal_JEP: Applied                                       4.1365      1.722      2.403      0.016       0.762       7.511
Journal_JEP: General                                       4.3057      2.148      2.005      0.045       0.097       8.515
Journal_JEP: Human Perception and Performance             -4.4976      1.298     -3.464      0.001      -7.042      -1.953
Journal_JEP: Learning, Memory, and Cognition               2.8982      0.969      2.992      0.003       1.000       4.797
Journal_Journal of Abnormal Psychology                     6.8945      0.753      9.160      0.000       5.419       8.370
Journal_Journal of Applied Psychology                     13.8493      1.238     11.187      0.000      11.423      16.276
Journal_Journal of Comparative Psychology                -26.4896      2.038    -13.000      0.000     -30.483     -22.496
Journal_Journal of Consulting and Clinical Psychology      2.6977      0.992      2.719      0.007       0.753       4.642
Journal_Journal of Counseling Psychology                   4.0915      1.341      3.051      0.002       1.463       6.720
Journal_

In [18]:
table0 = Table_Summary(r0)
table0 = table0.reindex(['Political Success','Subscriptions','JIF Percentile','Citable Items','Intercept','Adjusted R2'],level=0)
table0 = table0.reindex(['Coeff','CSE','p-val',''],level=1)
table0.columns = ['Base Model']

table1 = Table_Summary(r1)
table1 = table1.reindex(['Political Success','Subscriptions','Political Success*Subscriptions','Citable Items','JIF Percentile','Intercept','Adjusted R2'],level=0)
table1 = table1.reindex(['Coeff','CSE','p-val',''],level=1)
table1.columns = ['Full Model']

table = pandas.merge(table0,table1,on=['Variables','Statistics'],how='outer')
table = table.reindex(['Political Success','Subscriptions','Political Success*Subscriptions','Citable Items','JIF Percentile','Intercept','Adjusted R2'],level=0)
table.round(3)

Base Model  Full Model
Variables                       Statistics                        
Political Success               Coeff            1.008       0.883
                                CSE              0.164       0.148
                                p-val            0.000       0.000
Subscriptions                   Coeff           -0.622      -0.701
                                CSE              0.266       0.276
                                p-val            0.019       0.011
Political Success*Subscriptions Coeff              NaN      -0.175
                                CSE                NaN       0.069
                                p-val              NaN       0.012
Citable Items                   Coeff           -1.728      -1.857
                                CSE              0.551       0.551
                                p-val            0.002       0.001
JIF Percentile                  Coeff            0.652       0.658
                                CSE              0.513       0.516
                                p-val            0.204       0.202
Intercept                       Coeff           77.032      77.662
                                CSE              1.096       1.218
                                p-val            0.000       0.000
Adjusted R2                                      0.726       0.726

In [19]:
table.to_excel('Table-PS_robust1.xlsx',float_format="%.3f")